In [1]:
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
from PIL import Image
import os

#Part 1: Facial Expression Classification

## a) Data Preprocessing

###Dowbload from OpenML

In [3]:
!pip install openml

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 36.7 MB/s eta 0:00:00
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11717 sha256=5110bd6866308e8f579ddb5e6f89874db82e7fb3fc6e888d15a9f7d356587f2a
  Stored in directory: /root/.cache/pip/wheels/00/23/31/5e562fce1f95aabe57f2a7320d07433ba1cd152bcde2f6a002
Successfully built liac-arff


In [4]:

import openml


dataset = openml.datasets.get_dataset(30123)
dataset

OpenML Dataset
Name.........: QSAR-DATASET-FOR-DRUG-TARGET-CHEMBL1163104
Version......: 9
Format.......: ARFF
Upload Date..: 2016-07-15 04:12:58
Licence......: Public
Download URL.: https://api.openml.org/data/v1/download/3411463/QSAR-DATASET-FOR-DRUG-TARGET-CHEMBL1163104.arff
OpenML URL...: https://www.openml.org/d/30123
# of features: None

In [11]:

df = dataset.get_data(dataset_format="dataframe")

X, y, categorical_indicator, attribute_names = df

df

ValueError: too many values to unpack (expected 2)

In [10]:
X

,AAC,AECC,ALOGP,ALOGP2,AMR,AMW,ARR,ATS1e,ATS1i,ATS1m,...,nCXr,nDB,NdCH2,NddssS,NdO,NdsCH,NdsN,NdssC,NdssS,pXC50
0,1.682,10.037,3.884,15.085,105.742,8.778,0.767,3.876,3.969,3.685,...,0,1,0,0,1,0,0,0,1,5.0
1,1.496,10.400,4.705,22.138,100.976,8.671,0.821,3.803,3.913,3.514,...,0,0,0,0,0,0,0,0,0,5.0
2,1.476,15.333,5.907,34.897,153.448,6.944,0.512,4.416,4.543,3.928,...,0,1,0,0,1,0,0,1,0,5.0
3,1.983,9.880,4.908,24.087,99.228,12.274,0.444,3.792,3.884,3.775,...,0,1,0,0,1,0,0,1,0,5.0
4,1.772,12.294,3.917,15.346,129.707,8.237,0.459,4.189,4.302,3.832,...,0,2,0,0,1,2,0,1,0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,1.778,11.767,3.694,13.646,118.112,8.661,0.515,4.068,4.188,3.755,...,0,0,0,0,0,0,0,0,0,5.0
69,1.716,8.731,2.737,7.493,98.273,8.947,0.724,3.849,3.950,3.571,...,0,1,0,0,1,0,0,1,0,5.0
70,1.502,9.743,3.479,12.106,135.851,7.438,0.500,4.257,4.372,3.840,...,0,3,0,0,2,0,0,4,0,5.0
71,1.765,12.935,4.203,17.668,117.539,8.126,0.500,4.125,4.233,3.763,...,0,0,0,0,0,0,0,0,0,5.0


In [12]:
y

### preprocessing

In [14]:


# Resize images
img_size = 64  # Set desired size
resized_images = []
for index in X.index: # Iterate through rows of the DataFrame
    image_data = X.loc[index, 'pixels']  # Access the 'pixels' column by index
    image_data_array = np.fromstring(image_data, dtype=np.uint8, sep=' ').reshape(48, 48)  # Reshape directly
    img = Image.fromarray(image_data_array, 'L')  # Create PIL Image
    img = img.resize((img_size, img_size))
    resized_images.append(np.array(img))

X = np.array(resized_images)

# Normalize pixel values
X = X.astype('float32') / 255.0

# Split data
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


KeyError: 'pixels'

## b) Model Architecture
